# Aplicando modelagem de assuntos ao DHBB
Neste capítulo vamos explorar ferramentas de modelagem de assuntos e explorar aplicações ao DHBB. Como sempre, começamos com alguns imports familiares.

In [1]:
import os, glob, pickle
import spacy
from spacy import displacy
from sqlalchemy import create_engine
from dhbbmining import *
import ipywidgets as widgets
from tqdm import tqdm

Vamos também carregar o modelo de NLP para a língua portuguesa do Spacy:

In [2]:
!python -m spacy download pt_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [3]:
nlp = spacy.load("pt_core_news_sm")

Agora faremos alguns imports novos, particularmente da biblioteca [Gensim](https://radimrehurek.com/gensim), que nos oferece as ferramentas que necessitamos para modelagem de assuntos.

In [4]:
from string import punctuation
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec, word2vec

Para minimizar o uso de memória, vamos construir uma classe para representar o nosso corpus como um iterador, operando diretamente do banco de dados. Desta forma, ao fazer nossas análises, podemos carregar um documento por vez para alimentar os modelos, sem a necessidade de manter todo o corpus na memória, economizando memória RAM.

In [5]:
eng = create_engine("sqlite:///minha_tabela.sqlite")

class DHBBCorpus:
    def __init__(self, ndocs=10000):
        self.ndocs = min(7687,ndocs)
        self.counter = 1
    def __iter__(self):
        with eng.connect() as con:
            res = con.execute(f'select corpo from resultados limit {self.ndocs};')
            for doc in res:
                d = self.pre_process(doc[0])
                if self.counter%10 == 0:
                    print (f"Verbete {self.counter} de {6*self.ndocs}\r", end='')
                for s in d:
                    yield s
                self.counter += 1
    def pre_process(self, doc):
        n = nlp(doc, disable=['tagger', 'ner','entity-linker', 'textcat','entity-ruler','merge-noun-chunks','merge-entities','merge-subtokens'])
        results = []
        for sentence in n.sents:
            s = sentence.text.split()
            if not s:
                continue
            results.append([token.strip().strip(punctuation) for token in s if token.strip().strip(punctuation)])
        return results
        

Abaixo um pequeno exemplo de como a classe `DHBBCorpus` funciona:

In [6]:
DC = DHBBCorpus(5)
for f in DC:
    pass
    print(f)
    break
    

['«José', 'Machado', 'Coelho', 'de', 'Castro»', 'nasceu', 'em', 'Lorena', 'SP']


In [7]:
Word2Vec?

## Word2vec 
Vamos começar pelo treinamento de um modelo word2vec. Este modelo itera 6 vezez sobre o corpus logo, devemos ver o contador atingir 46122. Estas repetições são necessárias para permitir a 

In [8]:
if os.path.exists('dhbb.w2v'):
    model = Word2Vec.load('dhbb.w2v')
else:
    DC = DHBBCorpus()
    model = Word2Vec(sentences=DC, workers=32)
    model.save('dhbb.w2v')

### Explorando o modelo

In [9]:
for i, word in enumerate(model.wv.vocab):
    if i == 10:
        break
    print(word)

«José
Machado
Coelho
de
Castro»
nasceu
em
Lorena
SP
Estudou


In [10]:
model.wv['deputado']

array([ 1.213255  ,  0.8772738 , -0.06480797,  3.2624714 ,  2.7141786 ,
       -1.8534654 ,  2.2568564 ,  2.9633396 ,  2.872959  ,  0.17697811,
        0.12649159,  2.1954768 ,  0.69300246,  0.7808246 , -1.8608911 ,
       -1.9773395 , -6.7470145 ,  1.3309088 , -1.8495115 ,  1.4977074 ,
        4.106705  ,  0.18757463,  0.976982  ,  0.50778925, -1.3095387 ,
       -0.46490836,  1.3230817 , -1.0534579 , -3.3441863 , -4.0759525 ,
        0.20254904,  2.8288093 ,  0.0304382 ,  0.79879284, -0.27596825,
        1.0034108 ,  2.1574116 , -0.84378964, -2.6574318 ,  0.725499  ,
       -2.5952895 ,  0.6314094 , -2.789002  ,  5.0512304 , -1.6781955 ,
       -3.2558453 , -2.3100727 , -2.2926745 , -4.6876025 , -3.1457222 ,
        0.19902074, -0.6323914 ,  1.0432411 , -1.1918347 ,  4.250325  ,
       -2.759753  ,  2.7777524 ,  2.8528945 , -0.3918448 , -1.9812953 ,
        4.549228  , -6.3611536 ,  0.09752683,  0.37324116,  0.77719754,
       -3.6438515 , -3.4890673 , -2.3472724 ,  0.6998824 ,  3.61

In [11]:
len(model.wv.vocab)

38762

In [12]:
model.wv.most_similar(positive=['homem'], topn=20)

[('cidadão', 0.7166314125061035),
 ('dever', 0.7115763425827026),
 ('universo', 0.708888053894043),
 ('pensamento', 0.7076699733734131),
 ('nacionalismo', 0.7048879861831665),
 ('senso', 0.7020969390869141),
 ('inimigo', 0.6951218843460083),
 ('espírito', 0.691114068031311),
 ('sentimento', 0.6901904940605164),
 ('desafio', 0.6858540773391724),
 ('fenômeno', 0.6775267720222473),
 ('leitor', 0.6737638115882874),
 ('retrato', 0.6691586971282959),
 ('veículo', 0.6683763265609741),
 ('trabalhador', 0.6611828804016113),
 ('erro', 0.6600130200386047),
 ('povo”', 0.6572052836418152),
 ('negócio', 0.6528052091598511),
 ('homem”', 0.650109052658081),
 ('liberalismo', 0.6475819945335388)]

In [13]:
model.corpus_total_words

8713256

In [14]:
model.vocabulary?

### Visualizando os vetores de palavras

In [15]:
from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling


def reduce_dimensions(model):
    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = [] # positions in vector space
    labels = [] # keep track of words to label our data again later
    i = 0
    for word in model.wv.vocab:
        vectors.append(model.wv[word])
        labels.append(word)
        i+=1
        if i>200:
            break

    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)

    # reduce using t-SNE
    vectors = np.asarray(vectors)
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


x_vals, y_vals, labels = reduce_dimensions(model)

def plot_with_plotly(x_vals, y_vals, labels, plot_in_notebook=True):
    from plotly.offline import init_notebook_mode, iplot, plot
    import plotly.graph_objs as go

    trace = go.Scatter(x=x_vals, y=y_vals, mode='text', text=labels)
    data = [trace]

    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')

In [17]:
plot_with_plotly(x_vals, y_vals, labels)

In [18]:
model.wv.most_similar(positive=['congressista', 'política'], negative=['advogado'])

[('pragmática', 0.6115255355834961),
 ('“política', 0.6081854701042175),
 ('Resultante', 0.5997891426086426),
 ('político-institucional', 0.5906023979187012),
 ('político-social', 0.5800623893737793),
 ('inevitavelmente', 0.5741809010505676),
 ('nítida', 0.5726122856140137),
 ('persistente', 0.5723466873168945),
 ('induzida', 0.5718863010406494),
 ('vulnerabilidade', 0.5662615895271301)]

In [19]:
model.wv['congressista'] + model.wv['política']-model.wv['homem']

array([-1.1804812 ,  0.17904007,  2.7701726 ,  2.2788234 , -2.3939579 ,
       -1.6261489 ,  3.037264  , -1.5370936 ,  0.1686287 ,  1.7679174 ,
       -0.6206008 , -1.4379971 ,  0.9625195 ,  5.3786445 , -1.5540149 ,
       -0.00777411, -1.401691  , -4.2874527 , -0.82898766,  1.8527349 ,
        1.1073897 ,  0.4124229 , -1.7202759 , -1.9285159 ,  0.22249055,
       -0.1637733 ,  0.3505941 , -1.6477919 ,  3.5996528 , -1.884695  ,
        0.70941716,  2.4855223 ,  0.19784053,  1.0554858 , -2.0812526 ,
        0.25975254,  1.0439771 ,  0.40970242,  0.78404856, -2.2612848 ,
        1.7169695 ,  0.73526394,  1.4105549 ,  4.066178  , -0.40555423,
        1.8521545 ,  3.7054353 ,  1.3822607 , -4.056506  ,  1.9932911 ,
        0.89136565, -2.6281335 ,  3.6849546 ,  3.6759586 , -0.3893448 ,
       -0.93963474,  3.059989  ,  0.07065499, -0.69516784, -1.2888556 ,
        0.92977023, -1.6649121 ,  2.6862006 , -2.4784298 ,  1.8600826 ,
        0.779212  ,  2.1920981 , -1.9809456 , -2.688896  ,  0.60

## Word Movers Distance
Este método permite que calculemos a distância entre dois documentos mesmo que estes não contenham palavras em comum. Este método foi proposto no artigo ["From Word Embeddings To Document Distances"](http://jmlr.org/proceedings/papers/v37/kusnerb15.pdf).

In [41]:
DC = list(DHBBCorpus(5))
model.wv.wmdistance(document1=DC[1],document2=DC[2])

15.084479096899363

In [33]:
print(DC[1])
print(DC[2])

['Estudou', 'no', 'Ginásio', 'Diocesano', 'de', 'São', 'Paulo', 'e', 'bacharelou-se', 'em', '1910', 'pela', 'Faculdade', 'de', 'Ciências', 'Jurídicas', 'e', 'Sociais']
['Dedicando-se', 'à', 'advocacia', 'foi', 'promotor', 'público', 'em', 'Cunha', 'SP', 'e', 'depois', 'delegado', 'de', 'polícia', 'no', 'Rio', 'de', 'Janeiro', 'então', 'Distrito', 'Federal']
